# Compute Starlink satellites positions

In [1]:
import pandas as pd
df = pd.read_html("https://www.n2yo.com/satellites/?c=52",attrs={"id":"categoriestab"})[0]
df["Launch date"] = pd.to_datetime(df["Launch date"], format="%B %d, %Y")
df.dropna(inplace=True)
df["NORAD ID"] = df["NORAD ID"].astype(int) # Convert NORAD ID to integer
df.head()

,Name,NORAD ID,Int'l Code,Launch date,Period[minutes],Action
0,STARLINK 1068,44772,2019-074BM,2019-11-11,90.8,TRACK IT
1,STARLINK 1067,44771,2019-074BL,2019-11-11,90.8,TRACK IT
2,STARLINK 1065,44770,2019-074BK,2019-11-11,90.8,TRACK IT
3,STARLINK 1064,44769,2019-074BJ,2019-11-11,90.8,TRACK IT
4,STARLINK 1063,44768,2019-074BH,2019-11-11,90.8,TRACK IT


In [2]:
# Import 
from bs4 import BeautifulSoup
import requests

# Make a GET request to fetch the raw HTML content
tle = list()
for noradId in df["NORAD ID"].loc[0:1]:
    satinfo_URL = "https://www.n2yo.com/satellite/?s={0:d}".format(noradId)
    print(satinfo_URL)
    html_content = requests.get(satinfo_URL).text
    
    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    html_tle = soup.find("div", {"id": "tle"})
    tle_temp = "NORAD ID {0:d}\n".format(noradId) + "\n".join(html_tle.find("pre").text.replace("\r","").split("\n")[1:3])
    tle.append(tle_temp)

https://www.n2yo.com/satellite/?s=44772
https://www.n2yo.com/satellite/?s=44771


In [3]:
tle[0]

'NORAD ID 44772\n1 44772U 19074BM  19321.58334491  .00320810  00000-0  12893-2 0  9997\n2 44772  53.0073 141.1502 0001298  55.5500 297.1915 15.85056013   950'

## Compute satellite position from TLE

In [4]:
# Define moment of computation
import datetime
from astropy import time
t_comp = datetime.datetime(2019, 11, 17, 21, 14, 0, tzinfo=datetime.timezone(datetime.timedelta(hours=0)))
t_comp.tzname()
t_comp

datetime.datetime(2019, 11, 17, 21, 14, tzinfo=datetime.timezone.utc)

### Get topocentric position

In [9]:
import numpy as np
from astropy.coordinates import EarthLocation, BaseCoordinateFrame
from astropy import coordinates as coord
from pycraf import satellite

# Define observer location
location = EarthLocation(lon=17.5, lat=48, height=200)

# create a SatelliteObserver instance
sat_obs = satellite.SatelliteObserver(location)

# Get satellite position in Earth centered Intertial frame (ECI)
az, el, dist = sat_obs.azel_from_sat(tle[0], time.Time(t_comp))  
print('Azimuth: {:.1f}\nElevation: {:.1f}\nRange: {:.0f}'.format(az, el, dist)) 

Azimuth: 166.0 deg
Elevation: -50.3 deg
Range: 10193 km


### Get geocentric position in ECI frame

In [6]:
satname, sat = satellite.get_sat(tle[0])
now = [t_comp.year, t_comp.month, t_comp.day, t_comp.hour, t_comp.minute, t_comp.second]
satpos, satvel = sat.propagate(*now)
print("ECI\nX: {:.1f} km\nY: {:.1f} km\nZ: {:.1f} km".format(*satpos))

ECI
X: 2289.5 km
Y: 3336.7 km
Z: -5338.9 km


### Get geocentric position in ECEF frame

In [7]:
import astropy.units as u
gcrs = coord.SkyCoord(x=satpos[0]*u.km,
                      y=satpos[1]*u.km,
                      z=satpos[2]*u.km,
                      frame=coord.GCRS,
                      obstime=time.Time(t_comp),
                      representation_type="cartesian")
print("GCRS:\nX: {0:s}\nY: {1:s}\nZ: {2:s}".format(gcrs.x,gcrs.y,gcrs.z))

GCRS:
X: 2289.5155300754536 km
Y: 3336.684746687033 km
Z: -5338.929849804127 km


In [8]:
itrs = gcrs.transform_to(coord.ITRS)
#itrs = coord.cartesian_to_spherical(*itrs.data.xyz.value)
itrs.representation_type = 'spherical'
print("ITRS:\nLongitude: {0:s}\nLatitude: {1:s}\nHeight: {2:s}".format(itrs.lon,itrs.lat,itrs.distance-6378*u.km))

ITRS:
Longitude: 40.540698931408535 deg
Latitude: -52.77871401021861 deg
Height: 321.2177455567107 km
